# Data Exploration and Transformation Notebook for Vivino Wine Project

In [ ]:
# Install modules 
!pip install pandas
!pip install numpy
!pip install scipy

In [ ]:
# Import modules
import pandas as pd
import numpy as np
import scipy.stats as stats
from datetime import datetime

## Read data file

In [ ]:
# Read data from csv file
df = pd.read_csv('vivino_data_00_RAW.csv')
df.info()

## 1.1 Drop duplicated Rows

In [ ]:
# (1.1) Drop duplicated rows
print ('DROP DUPLICATES -- BEFORE ==', len(df))
df.drop_duplicates(inplace=True)
print ('DROP DUPLICATES -- AFTER ==', len(df))

## 1.2 Drop rows with wine_type != Red or White wine

In [ ]:
# (1.2) Drop rows with wine_type != Red or White wine
print ('DROP ROWS WITH wine_type != Red wine or White wine -- BEFORE ==', len(df))
df = df[(df.wine_type == 'Red wine') | (df.wine_type == 'White wine')]
print ('DROP ROWS WITH wine_type != Red wine or White wine -- AFTER ==', len(df))

## 1.3 Add new wine_age feature + Drop rows with wines older than 25 years

In [ ]:
# (1.3) Drop rows with wines older than 25 years 
print ('NEW FIELD ADDED: wine_age')
print ('DROP ROWS WITH wine_age > 25 -- BEFORE ==', len(df))
df['wine_age'] = datetime.today().year - df.year
df = df[(df.wine_age <= 25)]
print ('DROP ROWS WITH wine_age > 25 -- AFTER ==', len(df))

## 1.4 Drop rows with price < 0 or >= 120

In [ ]:
# (1.4) Drop rows with price < 0 or >= S$120
print ('DROP ROWS WITH price < 0 or price >= 120 -- BEFORE ==', len(df))
df = df[(df.price < 120) & (df.price >= 10)]
print ('DROP ROWS WITH price < 0 or price >= 120 -- AFTER ==', len(df))

## 2.1 For rows with NA / empty value in grape -- replace value with "Unspecified"

In [ ]:
# (2.1) For rows with NA / empty value in grape -- replace value with "Unspecified"
print ('FILL NA/EMPTY grape CELLS WITH VALUE = UNSPECIFIED')
df.grape.fillna('Unspecified', inplace=True)

## 2.2 For rows with "-1" value in taste characteristic -- replace value with category mean (grape) for that characteristic

Example: If the dry_sweet taste note of a merlot grape wine is -1, we will replace it with the mean of dry_sweet values of all merlot grape wines (mean is calculated only from those merlot grapes with dry_sweet value >= 0)

In [ ]:
# (2.2) For rows with "-1" value in taste characteristic -- replace value with category mean (grape)
print ('FILL TASTE_PROFILE CELLS OF VALUE < 0 WITH CATEGORY MEAN\n')
columns = [ 'light_bold', 'smooth_tannic', 'dry_sweet', 'soft_acidic' ]

for c in columns:
    df1 = df[(df[c] >= 0)]
    df2 = df[(df[c] < 0)]

    # Find the mean of the taste characteristic (e.g. light_bold, dry_sweet) for each type grape
    print ('LINES AFFECTED FOR', c, ' =', len(df2))
    g = df1.groupby(['grape'])
    means = g[c].mean()

    # When we encounter a "-1" taste characteristic value, we look up the means table, to find the mean value 
    # we stored earlier... we replace the "-1" value with this means value
    df2[c] = df2.grape.map(means)
    
    # If the means value isn't found in the lookup table, we replace the "-1" value with the characteristic's 
    # column mean value (less accurate)
    df2[c].fillna(df1[c].mean(), inplace=True)

    df.update(df2, join='left')

## 2.3 Write cleaned data to csv file

In [ ]:
# Write cleaned data to csv file
# The code below is for writing files to your computer. If writing files to IBM Cloud Storage,
# use the code fragment provided in the IBM Cloud for reading the data asset from IBM Cloud Storage.
print ('WRITE CLEANED DATAFRAME TO FILE')
df.to_csv('vivino_data_00-1_CLEANED.csv', index=False)

## 3.1 Add geographic region field

In [ ]:
# (3.1) Add geographic region field
print ('NEW FIELD ADDED: geo_region')
geo_region = {'W Europe': ['France', 'Germany', 'United Kingdom'],
            'S Europe': ['Greece', 'Italy', 'Portugal', 'Spain'],
            'C/E Europe': ['Austria', 'Bulgaria', 'Hungary', 'Moldova', 'North Macedonia', 'Romania', 'Slovenia'],
            'N America': ['Canada', 'United States'],
            'S America': ['Argentina', 'Brazil', 'Chile', 'Uruguay'],
            'Oceania': ['Australia', 'New Zealand'],
            'Africa': ['South Africa'],
            'C/W Asia': ['Georgia', 'Israel', 'Lebanon'],
            'E Asia': ['China']}
lookup_table = {}
for k, v in geo_region.items():
    for c in v:
        lookup_table[c] = k
df['geo_region'] = df['country'].map(lookup_table)

## 3.2 Write transformed data to csv file

In [ ]:
# (3.2) Write transformed data to csv file
print ('WRITE TRANSFORMED DATAFRAME TO FILE')
df.to_csv('vivino_data_00-2_TRANSFORMED.csv', index=False)

## 3.3 Create stratified random sample

In [ ]:
# (3.3) Create stratified random sample
print ('CREATE STRATIFIED RANDOM SAMPLE -- GROUPED BY COUNTRY')
sample_size = 10000
df = df.groupby('country', group_keys=False).apply(lambda x: x.sample(int(np.rint(sample_size*len(x)/len(df))))).sample(frac=1).reset_index(drop=True)
print ('Required Sample Size =', sample_size)
print ('Actual Sample Size =', len(df))

## 3.4 Write stratified random sample to csv file

In [ ]:
# (3.4) Write stratified random sample to csv file
print ('WRITE STRATIFIED RANDOM SAMPLE DATAFRAME TO FILE (ALL FIELDS INTACT)')
df.to_csv('vivino_data_00-3_ALL_FIELDS.csv', index=False)

## 4.1 Drop some columns to get under 1 MB (IBM Cloud sample limit)


In [ ]:
# (4.1) Drop some columns to get under 1 MB (IBM Cloud sample limit)
df.drop(['winery', 'wine_name', 'region', 'taste_mentions', 'taste_notes', 'wine_url'], axis=1, inplace=True)
print ('DROPPED FIELDS: winery, wine_name, region, taste_mentions, taste_notes, wine_url')

## 4.2 Remove the word "wine" from wine_type to reduce clutter

In [ ]:
# (4.2) Remove the word "wine" from wine_type to reduce clutter
df['wine_type'] = df['wine_type'].str.replace('[ ](w|W)ine', '', regex=True)

## 4.3 Write stratified random sample to csv file

In [ ]:
# 4.3 Write stratified random sample to csv file
print ('WRITE STRATIFIED RANDOM SAMPLE DATAFRAME TO FILE')
df.to_csv('vivino_data_00-4_FINAL.csv', index=False)
df.info()

# End of Notebook